In [35]:
import pandas as pd

In [36]:
# Dataset downloaded from Kaggle.com (https://www.kaggle.com/datasets/eswaranmuthu/u-s-economic-vital-signs-25-years-of-macro-data) and was
# originally published by FRED (Federal Reserve Economic Data - https://fred.stlouisfed.org/)

# Read in the data from .csv
econ_data = pd.read_csv('macro_data_25yrs.csv')

# Display the DataFrame
econ_data.head(3)

,Date,M2_Money_Supply,10Y Treasury Yield,Fed Funds Rate,CPI,Inflation_Rate_%,SOFR
0,2018-04-03,13993.9,2.87,1.69,250.227,2.470996,1.83
1,2018-04-04,13993.9,2.87,1.69,250.227,2.470996,1.74
2,2018-04-05,13993.9,2.87,1.69,250.227,2.470996,1.75


In [37]:
# Convert the 'Date' column from string to datetime
econ_data['Date'] = pd.to_datetime(econ_data['Date'])

In [38]:
# Group the data by Year and Month_Name using the median value for the month
econ_data = pd.DataFrame(econ_data.groupby([econ_data['Date'].dt.year, econ_data['Date'].dt.month])[['M2_Money_Supply', '10Y Treasury Yield', 'Fed Funds Rate', 'CPI', 'Inflation_Rate_%', 'SOFR']].median())

# Rename the columns in the index (brought in by the groupby) to break the redundancy
econ_data.index.names = ['Year', 'Month']

#Sanity check
econ_data.head(3)

M2_Money_Supply  10Y Treasury Yield  Fed Funds Rate      CPI  \
Year Month                                                                 
2018 4              13993.9                2.87            1.69  250.227   
     5              14049.6                2.98            1.70  250.792   
     6              14109.9                2.91            1.82  251.018   

            Inflation_Rate_%   SOFR  
Year Month                           
2018 4              2.470996  1.745  
     5              2.781922  1.730  
     6              2.807551  1.870

In [39]:
# Coerce the Index columns into regular DataFrame columns
econ_data.reset_index(inplace=True)

# Sanity check
econ_data.head(3)

,Year,Month,M2_Money_Supply,10Y Treasury Yield,Fed Funds Rate,CPI,Inflation_Rate_%,SOFR
0,2018,4,13993.9,2.87,1.69,250.227,2.470996,1.745
1,2018,5,14049.6,2.98,1.70,250.792,2.781922,1.730
2,2018,6,14109.9,2.91,1.82,251.018,2.807551,1.870


In [40]:
# Rename the Month values from numerics into human readable month names
econ_data['Month'] = econ_data['Month'].replace({1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 
                11:'November', 12:'December'})

# Sanity check
econ_data.head(3)

,Year,Month,M2_Money_Supply,10Y Treasury Yield,Fed Funds Rate,CPI,Inflation_Rate_%,SOFR
0,2018,April,13993.9,2.87,1.69,250.227,2.470996,1.745
1,2018,May,14049.6,2.98,1.70,250.792,2.781922,1.730
2,2018,June,14109.9,2.91,1.82,251.018,2.807551,1.870


In [41]:
# Print a list of column names
econ_data.columns

Index(['Year', 'Month', 'M2_Money_Supply', '10Y Treasury Yield',
       'Fed Funds Rate', 'CPI', 'Inflation_Rate_%', 'SOFR'],
      dtype='object')

In [42]:
# Use the column names above to target the last 3 columns and coerce them to be two decimal places using a loop controlled by a sliced list
for i in econ_data.columns[5:]:
    econ_data[i] = econ_data[i].round(2)

# Sanity check
econ_data.tail(3)

,Year,Month,M2_Money_Supply,10Y Treasury Yield,Fed Funds Rate,CPI,Inflation_Rate_%,SOFR
84,2025,April,21862.4,4.28,4.33,320.32,2.33,4.35
85,2025,May,21942.0,4.42,4.33,320.58,2.38,4.30
86,2025,June,21942.0,4.42,4.33,320.58,2.38,4.29


In [43]:
# Rename the columns to more desirable and human interpretable values
econ_data.columns = ['Year', 'Month', 'M2_Money_Supply_in_Billions', '10Y_Treasury_Yield_%', 'Fed_Funds_Rate_%', 'Consumer_Price_Index', 
                     'Inflation_Rate_%', 'Secured_Overnight_Financing_Rate']

# Sanity check
econ_data.head(3)

,Year,Month,M2_Money_Supply_in_Billions,10Y_Treasury_Yield_%,Fed_Funds_Rate_%,Consumer_Price_Index,Inflation_Rate_%,Secured_Overnight_Financing_Rate
0,2018,April,13993.9,2.87,1.69,250.23,2.47,1.74
1,2018,May,14049.6,2.98,1.70,250.79,2.78,1.73
2,2018,June,14109.9,2.91,1.82,251.02,2.81,1.87


In [ ]:
# Reset the index back to year and month now that the month values have been coerced (Indexes are not hashable)
econ_data.set_index(['Year', 'Month'], inplace=True)

In [45]:
# Find statistical correlations of numberic columns in the DataFrame
econ_data.corr()  

,M2_Money_Supply_in_Billions,10Y_Treasury_Yield_%,Fed_Funds_Rate_%,Consumer_Price_Index,Inflation_Rate_%,Secured_Overnight_Financing_Rate
M2_Money_Supply_in_Billions,1.000000,0.375080,0.292711,0.826628,0.566684,0.281296
10Y_Treasury_Yield_%,0.375080,1.000000,0.912087,0.791829,0.170698,0.907554
Fed_Funds_Rate_%,0.292711,0.912087,1.000000,0.752711,-0.063533,0.999314
Consumer_Price_Index,0.826628,0.791829,0.752711,1.000000,0.344000,0.743981
Inflation_Rate_%,0.566684,0.170698,-0.063533,0.344000,1.000000,-0.078343
Secured_Overnight_Financing_Rate,0.281296,0.907554,0.999314,0.743981,-0.078343,1.000000


In [ ]:
# Sanity check
econ_data.tail(5)

M2_Money_Supply_in_Billions  10Y_Treasury_Yield_%  \
Year Month                                                         
2025 February                      21613.5                  4.45   
     March                         21706.4                  4.28   
     April                         21862.4                  4.28   
     May                           21942.0                  4.42   
     June                          21942.0                  4.42   

               Fed_Funds_Rate_%  Consumer_Price_Index  Inflation_Rate_%  \
Year Month                                                                
2025 February              4.33                319.77              2.81   
     March                 4.33                319.62              2.41   
     April                 4.33                320.32              2.33   
     May                   4.33                320.58              2.38   
     June                  4.33                320.58              2.38   

               Secured_Overnight_Financing_Rate  
Year Month                                       
2025 February                              4.34  
     March                                 4.33  
     April                                 4.35  
     May                                   4.30  
     June                                  4.29